In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
                app_id="ZGCsmM0t5c9VMLRJ",                                 
                app_secret="T5eQ70WLmOWDpVvV7jicJtspuBR2A361nzVGmR8xTF4=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Hilman-D")            

account_info = deri.get_account_info()

account_info    

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [4]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
limit=1,
normalized=True,)

In [5]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,136481,1746723600,38.0,38.0,37.25,37.25,97600,3668975.0


In [6]:
session.set_keyspace('data_stock')

In [7]:
session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open_price float,
        high_price float,
        low_price float,
        close_price float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

✅ Keyspace และ Table พร้อมใช้งาน!


In [8]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
import pandas as pd

try:
    symbols_df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settread_data.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'get_stock_name' กรุณาตรวจสอบชื่อไฟล์และตำแหน่งให้ถูกต้อง")
    raise  # หยุดโปรแกรมทันทีโดยแสดง error


# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_financial_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_financial_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล TRUBB ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น TRUBB สำเร็จ!

📦 กำลังโหลดข้อมูล TRUE ...
✅ เพิ่มข้อมูล 530 รายการของหุ้น TRUE สำเร็จ!

📦 กำลังโหลดข้อมูล TRV ...
✅ เพิ่มข้อมูล 684 รายการของหุ้น TRV สำเร็จ!

📦 กำลังโหลดข้อมูล TSC ...
✅ เพิ่มข้อมูล 723 รายการของหุ้น TSC สำเร็จ!

📦 กำลังโหลดข้อมูล TSE ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น TSE สำเร็จ!

📦 กำลังโหลดข้อมูล TSI ...
✅ เพิ่มข้อมูล 724 รายการของหุ้น TSI สำเร็จ!

📦 กำลังโหลดข้อมูล TSR ...
✅ เพิ่มข้อมูล 727 รายการของหุ้น TSR สำเร็จ!

📦 กำลังโหลดข้อมูล TSTE ...
✅ เพิ่มข้อมูล 536 รายการของหุ้น TSTE สำเร็จ!

📦 กำลังโหลดข้อมูล TSTH ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น TSTH สำเร็จ!

📦 กำลังโหลดข้อมูล TTA ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น TTA สำเร็จ!

📦 กำลังโหลดข้อมูล TTB ...
✅ เพิ่มข้อมูล 732 รายการของหุ้น TTB สำเร็จ!

📦 กำลังโหลดข้อมูล TTCL ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น TTCL สำเร็จ!

📦 กำลังโหลดข้อมูล TTI ...
✅ เพิ่มข้อมูล 362 รายการของหุ้น TTI สำเร็จ!

📦 กำลังโหลดข้อมูล TTLPF ...
✅ เพิ่มข้อมูล 672 รายการของหุ้น TTLP

In [11]:
stock = "TRUBB"
rows = session.execute(f"SELECT * FROM candlestick_data WHERE symbol = '{stock}'")

In [12]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()

,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,TRUBB,2022-05-09,2.94,3.00,2.92,2.98,5646778.0,1915321
1,TRUBB,2022-05-10,2.94,3.00,2.84,2.92,18247792.0,6273709
2,TRUBB,2022-05-11,2.98,3.04,2.92,2.96,12397659.0,4174997
3,TRUBB,2022-05-12,2.68,2.98,2.64,2.96,37857000.0,13736868
4,TRUBB,2022-05-13,2.62,2.72,2.60,2.70,25754214.0,9732420


In [14]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()